<a href="https://colab.research.google.com/github/akshaysutar8860/natural_launguage_processing/blob/main/fakenewsclassifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [184]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
import re
import nltk
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics
from sklearn.linear_model import PassiveAggressiveClassifier

In [128]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [129]:
frame1=pd.read_csv('https://raw.githubusercontent.com/krishnaik06/Fake-News-Classifier/master/fake-news/test.csv')

In [130]:
frame2=pd.read_csv('https://raw.githubusercontent.com/krishnaik06/Fake-News-Classifier/master/fake-news/submit.csv')

In [131]:
frame1.dropna()
frame2.dropna()

,id,label
0,20800,0
1,20801,1
2,20802,0
3,20803,1
4,20804,1
...,...,...
5195,25995,0
5196,25996,1
5197,25997,0
5198,25998,1


In [132]:
frame2.shape

(5200, 2)

In [133]:
frame1.head(2)

,id,title,author,text
0,20800,"Specter of Trump Loosens Tongues, if Not Purse...",David Streitfeld,"PALO ALTO, Calif. — After years of scorning..."
1,20801,Russian warships ready to strike terrorists ne...,NaN,Russian warships ready to strike terrorists ne...


In [134]:

# #training data
# train_X=frame1.iloc[0:4000,0:4]
# val_X=frame1.iloc[4000:5200,0:4]

# #**********
# train_y=frame2.iloc[0:4000,1:2]
# val_y=frame2.iloc[4000:5200,1:2]


# #validation data
# train_y=train_y['label']
# val_y=val_y['label']

In [146]:
messeges=frame1.copy()
stemer=PorterStemmer()
cv=CountVectorizer(ngram_range=(1,3))

In [147]:
corpus=[]
for i in range(len(messeges)):
  review=re.sub('[^a-zA-Z]',' ',str(messeges['title'][i]))
  review=review.lower()
  review=review.split()
  review=[stemer.stem(word) for word in review if not word in set(stopwords.words('english'))]
  review=' '.join(review)
  corpus.append(review)


In [148]:
corpus[0:5]

['specter trump loosen tongu purs string silicon valley new york time',
 'russian warship readi strike terrorist near aleppo',
 'nodapl nativ american leader vow stay winter file lawsuit polic',
 'tim tebow attempt anoth comeback time basebal new york time',
 'keiser report meme war e']

In [156]:
X=cv.fit_transform(corpus).toarray()

In [157]:
X.shape

(5200, 67020)

In [158]:
y=frame2['label']

In [197]:
train_X,val_X,train_y,val_y=train_test_split(X,y,test_size=0.2,random_state=0)

In [198]:
train_X.shape

(4160, 67020)

In [199]:
train_y.shape

(4160,)

In [200]:
cv.get_feature_names()[67000:67020]

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


['zuckerberg priscilla chan',
 'zuckerberg reveal',
 'zuckerberg reveal facebook',
 'zulu',
 'zulu bridg',
 'zulu bridg mumbai',
 'zum',
 'zum babywort',
 'zum babywort de',
 'zum pferdestehlen',
 'zum pferdestehlen sucht',
 'zumwalt',
 'zumwalt obama',
 'zumwalt obama russian',
 'zurich',
 'zurich new',
 'zurich new york',
 'zustimmung',
 'zustimmung gewinnen',
 'zustimmung gewinnen wenn']

In [201]:
cv.get_params()

{'analyzer': 'word',
 'binary': False,
 'decode_error': 'strict',
 'dtype': numpy.int64,
 'encoding': 'utf-8',
 'input': 'content',
 'lowercase': True,
 'max_df': 1.0,
 'max_features': None,
 'min_df': 1,
 'ngram_range': (1, 3),
 'preprocessor': None,
 'stop_words': None,
 'strip_accents': None,
 'token_pattern': '(?u)\\b\\w\\w+\\b',
 'tokenizer': None,
 'vocabulary': None}

In [202]:
sample=pd.DataFrame(train_X,columns=cv.get_feature_names())

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [203]:
sample

,aaron,aaron hernandez,aaron hernandez bodi,aaron hernandez murder,aaron klein,aaron klein lead,ab,ab reform,ab reform put,abandon,abandon climat,abandon climat chang,abandon espn,abandon espn drove,abandon evil,abandon evil queen,abandon fan,abandon fan base,abandon hillari,abandon hillari except,abandon polici,abandon polici coverag,abandonada,abandonada video,abba,abba aid,abba aid move,abba gestur,abba gestur critic,abba vow,abba vow reach,abbey,abbey new,abbey new york,abbi,abbi martin,abbi martin expos,abc,abc face,abc face boycott,...,zone syria save,zone two,zone two place,zoo,zoo gorilla,zoo gorilla drink,zoo new,zoo new york,ztech,zu,zu verwenden,zuckerberg,zuckerberg breitbart,zuckerberg buck,zuckerberg buck tide,zuckerberg control,zuckerberg control new,zuckerberg get,zuckerberg get hoodwink,zuckerberg priscilla,zuckerberg priscilla chan,zuckerberg reveal,zuckerberg reveal facebook,zulu,zulu bridg,zulu bridg mumbai,zum,zum babywort,zum babywort de,zum pferdestehlen,zum pferdestehlen sucht,zumwalt,zumwalt obama,zumwalt obama russian,zurich,zurich new,zurich new york,zustimmung,zustimmung gewinnen,zustimmung gewinnen wenn
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4155,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4156,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4157,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4158,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


# **Naive bias algorithm**

In [204]:
model=MultinomialNB(alpha=0.2)
model.fit(train_X,train_y)
predict=model.predict(val_X)


In [205]:
score=metrics.accuracy_score(val_y,predict)
cm=metrics.confusion_matrix(val_y,predict)

In [206]:
cm

array([[336, 122],
       [170, 412]])

In [207]:
score

0.7192307692307692

In [211]:
model.coef_[0]

array([0.11436436, 0.05877872, 0.00186203, ..., 0.05962251, 0.05962251,
       0.05962251])

In [212]:
fetures=cv.get_feature_names()[:20]

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [217]:

#most real words
sorted(zip(model.coef_[0],fetures),reverse=True)[:20]

[(0.11436435667471534, 'aaron'),
 (0.06181792261565405, 'abandon climat chang'),
 (0.06181792261565405, 'abandon climat'),
 (0.058778717432170446, 'aaron hernandez'),
 (0.056916687554626075, 'aaron hernandez murder'),
 (0.05558563924254489, 'aaron klein lead'),
 (0.05558563924254489, 'aaron klein'),
 (0.04209839691792612, 'abandon hillari except'),
 (0.04209839691792612, 'abandon hillari'),
 (0.0018620298775443747, 'aaron hernandez bodi'),
 (0.0, 'abandon fan base'),
 (0.0, 'abandon fan'),
 (-0.02898127636990322, 'abandon'),
 (-0.08364126546833778, 'abandon evil queen'),
 (-0.08364126546833778, 'abandon evil'),
 (-0.10434513829317274, 'abandon espn drove'),
 (-0.10434513829317274, 'abandon espn'),
 (-0.19952765060312455, 'ab reform put'),
 (-0.19952765060312455, 'ab reform'),
 (-0.19952765060312455, 'ab')]

In [216]:

#most fake words
sorted(zip(model.coef_[0],fetures))[:20]

[(-0.19952765060312455, 'ab'),
 (-0.19952765060312455, 'ab reform'),
 (-0.19952765060312455, 'ab reform put'),
 (-0.10434513829317274, 'abandon espn'),
 (-0.10434513829317274, 'abandon espn drove'),
 (-0.08364126546833778, 'abandon evil'),
 (-0.08364126546833778, 'abandon evil queen'),
 (-0.02898127636990322, 'abandon'),
 (0.0, 'abandon fan'),
 (0.0, 'abandon fan base'),
 (0.0018620298775443747, 'aaron hernandez bodi'),
 (0.04209839691792612, 'abandon hillari'),
 (0.04209839691792612, 'abandon hillari except'),
 (0.05558563924254489, 'aaron klein'),
 (0.05558563924254489, 'aaron klein lead'),
 (0.056916687554626075, 'aaron hernandez murder'),
 (0.058778717432170446, 'aaron hernandez'),
 (0.06181792261565405, 'abandon climat'),
 (0.06181792261565405, 'abandon climat chang'),
 (0.11436435667471534, 'aaron')]

# **aggressive classifier algorithm**

In [209]:
model=PassiveAggressiveClassifier(n_iter_no_change=50)
model.fit(train_X,train_y)
pred=model.predict(val_X)
score=metrics.accuracy_score(val_y,pred)
cm=metrics.confusion_matrix(val_y,pred)

In [210]:
score   #less accuracy

0.7423076923076923